In [ ]:
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("data").load_data()
# document 만드는게 나오진 않겠지?
parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)

index = VectorStoreIndex.from_documents(documents)
index = VectorStoreIndex.from_documents(documents=documents, transformations=[text_splitter])
# chunk를 만들때 어떻게 변화가 생기는지 체크하면서 외우기

node_id = index.index_struct.nodes_dict
# 키(key)는 노드 ID, 값(value)은 해당 노드 객체

for key, value in node_id.items():
    print(key, value)
    node_example_id = value
    break
    # 이것도 코드 자체는 첫번째것만 찍어내는 예제 케이스
    
index.vector_store.data.embedding_dict[node_example_id]
# 특정 노드의 임베딩 벡터 표현을 조회하는 코드

In [ ]:
# 그냥 이런식으로 할수도 있구나 정도로 파악해두면 될 듯.
from llama_index.core.node_parser import SentenceSplitter
import tiktoken

parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
# you can change chunk_size, chunk_overlap

nodes = parser.get_nodes_from_documents(documents)

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

encoded_text = tokenizer.encode(nodes[0].text)

decoded_text = tokenizer.decode(encoded_text)

In [ ]:
import textwrap

wrapper = textwrap.TextWrapper(width=80)

with open(path_to_txt, 'r', encoding='utf-8') as file:
    for line in file: # 텍스트 파일을 한 줄씩 읽으면서
        if 'paul graham' in line.lower(): # 특정 키워드를 찾아서
            formatted_text = wrapper.fill(line.strip()) # 그 줄을 80글자 단위로 줄바꿈 처리
            # formatted_text = textwrap.fill(line.strip(), width=80) 요런식으로 할 수도
            print(formatted_text.split('.')[-1]) # 마지막 문장만 출력

# 나오진 않을 듯? 그래도 읽어는 보기

In [ ]:
"""
    index.as_retriever() : 검색만 수행, 결과를 직접 활용하거나 다른 모델에 전달할 때
    index.as_query_engine() : 검색 + 답변 생성 / 질문에 대한 완성된 답변을 얻고 싶을 때
"""
retriever = index.as_retriever()
ret_passages = retriever.retrieve("Who is the author?")

query_engine = index.as_query_engine()
context_str = query_engine.query("Who is the author?")

In [ ]:
documents = reader.load_data(city_names, auto_suggest=False)
index = VectorStoreIndex.from_documents(documents) # VectorStore 만드는 기본 Flow

docu = Document(text=data_text, id_="new_doc_id")
# 새 문서를 생성

""" 입력 """
index.insert(docu)
# 기존 인덱스(index)에 새 문서를 삽입.문서가 임베딩 벡터로 변환되어 벡터 저장소에 추가

docu.set_content(value="Natural diamonds were (and are) formed (thousands of million years ago) in the upper mantle of Earth in metallic melts at temperatures of 2,000–6,000 °C and at pressures of 8–9 GPa.")
# 기존 docu 객체의 내용을 새로운 텍스트로 교체하는 코드
# vector_store에는 단순히 벡터만 있는 게 아니라, 
# docu라는 문서 객체와 그 내용이 임베딩과 함께 연결된 형태로 저장 된단다.

""" 갱신 """
output = index.update_ref_doc( # 인덱스에 참조된 문서를 갱신하는 메서드
    docu,
    update_kwargs={"delete_kwargs": {"delete_from_docstore": True}},
    # 기존 문서를 docstore에서 삭제한 뒤 새로운 내용으로 교체하라는 의미.
)

output = index.refresh_ref_docs([docu])
# 전달된 문서 리스트를 기반으로 내용을 다시 임베딩하고 vector store에 반영
# 기존 문서 ID는 유지되지만, 텍스트와 임베딩은 새 값으로 교체
# 옵션을 따로 주지 않아도 기본적으로 문서 내용을 최신 상태로 갱신합니다.

print(index.ref_doc_info.keys())
# 인덱스에 등록된 참조 문서(ref docs)들의 ID 목록이 출력됨.

""" 삭제 """
id = docu.doc_id

index.delete_ref_doc(id, delete_from_docstore=True)
#벡터 저장소(Vector Store)에서 임베딩 + 원본 객체까지 삭제

In [ ]:
""" Synthesizer를 활용하는 예제 
    response_mode를 통해 답변 스타일 지정 가능
        "compact" : 응답을 짧고 간결하게 요약
        "tree_summarize" : 여러 문서 조각을 트리 구조로 요약하여 응답을 생성.
                            긴 문서나 복잡한 질문에 적합.
        "refine" : 초기 응답을 생성한 뒤, 추가 문서 조각을 순차적으로 반영하여 점진적으로 개선.
                    정밀한 답변이 필요할 때 사용.
        "simple" : 가장 기본적인 모드, 검색된 내용을 단순히 합성.
"""

class StandardQueryEngine(CustomQueryEngine):
    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj

retriever = index.as_retriever()
synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = StandardQueryEngine(
    retriever=retriever, response_synthesizer=synthesizer
)


In [ ]:
""" Synthesizer 대신 llm을 직접 부르는 코드 """

from llama_index.llms.openai import OpenAI

# 내가 직접 포맷을 정의해서 프롬프트를 보낼 수 있다는 장점
simple_qa_prompt = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

class OurCustomQueryEngine(CustomQueryEngine):

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer # 쓰이지 않음. 불필요한걸 왜 남겼을까?
    llm: OpenAI
    qa_prompt: PromptTemplate = simple_qa_prompt

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            self.qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)